**Initial setup**

General purpose imports

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Imports for exploratory data analysis only

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns, folium
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from wordcloud import WordCloud, STOPWORDS

Basic settings

In [ ]:
seed = 42
#sample_n = 1000
sns.set(style = 'ticks') # 'darkgrid')
palette = {'negative': 'red', 'neutral': 'orange', 'positive': 'chartreuse'} # 'cyan' 'magenta'

Data import

In [ ]:
df0 = pd.read_csv('../input/twitter-airline-sentiment/Tweets.csv')

**Exploratory data analysis**

Basics

In [ ]:
print(df0.shape, df0.dtypes, sep = '\n')

In [ ]:
#df0.describe
#for x in df0:
#    print(x, df0[x].head(), sep = '\n')
print(df0.head())

In [ ]:
print(df0.airline_sentiment.value_counts())
ax = sns.countplot(y = 'airline_sentiment', data = df0, order = palette.keys(), palette = palette)

In [ ]:
print(df0.negativereason.value_counts())
ax0 = sns.countplot(y = 'negativereason', data = df0, order = df0.negativereason.value_counts().index)

In [ ]:
print(df0[['airline_sentiment', 'airline_sentiment_gold', 'negativereason', 'negativereason_gold', 'tweet_id']].fillna('-').groupby(['airline_sentiment', 'airline_sentiment_gold', 'negativereason', 'negativereason_gold']).agg('count'))

In [ ]:
## “What's in a name”
#print(df0.name.value_counts())
chart1, ax1 = plt.subplots()
sns.lineplot(data = df0.name.value_counts().cumsum().reset_index(drop = True))
ax1.set(xlabel = 'Accounts #', ylabel = 'Tweets #')
#sns.barplot(x = 'index', y = 'name', data = df0.name.value_counts().head(200).reset_index())
chart2, ax2 = plt.subplots()
sns.distplot(df0.name.value_counts(), kde = False, rug = True, axlabel = 'Tweets # per account')
ax2.set(ylabel = 'Accounts #')
df6 = df0.name.value_counts().head(20).reset_index().merge(df0, left_on = 'index', right_on = 'name', suffixes = ('_x', '')).sort_values('name_x', ascending = False)
df6 = pd.crosstab(df6.name, df6.airline_sentiment, margins = True).sort_values('All', ascending = False)
print('Top accounts', df6, sep = '\n')#.to_string)
del(df6)

In [ ]:
df0['tweet_created'] = pd.to_datetime(df0['tweet_created'], utc = True, errors = 'coerce')
df0['UTC_day_of_week'] = df0['tweet_created'].dt.dayofweek
df0['UTC_date'] = df0['tweet_created'].dt.date

In [ ]:
df3 = df0[['UTC_date', 'airline_sentiment', 'airline']]
df3['n'] = 1
df4 = df3.groupby(['UTC_date', 'airline_sentiment']).agg('count').reset_index()
chart3, ax3 = plt.subplots()
sns.lineplot(x = 'UTC_date', y = 'n', data = df4, hue = 'airline_sentiment', hue_order = palette.keys(), palette = palette)
ax3.set_title('Overall airline sentiment over time')
sns.relplot(col = 'airline', x = 'UTC_date', y = 'n', data = df3.groupby(['airline', 'UTC_date', 'airline_sentiment']).agg('count').reset_index(), kind = 'line', hue = 'airline_sentiment', hue_order = palette.keys(), palette = palette)
del(df3)

Location analysis

In [ ]:
v1 = df0['tweet_coord'].notna()
df1 = pd.DataFrame()
df1[['lat', 'lon']] = pd.DataFrame(df0['tweet_coord'][v1].apply(lambda x: [pd.eval(x)[0], pd.eval(x)[1]]).values.tolist(), index = df0.index[v1])
df0 = df0.merge(df1, left_index = True, right_index = True, how = 'left', validate = '1:1') # df0.join(df1, how = 'left')

for x in ['tweet_coord', 'tweet_location', 'user_timezone']:
    df0[x+ '_f'] = 0
    df0[x + '_f'][df0[x].notna() & (df0[x] != '[0.0, 0.0]')] = 1
    print(df0[x + '_f'].value_counts())

#sns.set(style = 'darkgrid')
#sns.scatterplot(x = 'lon', y = 'lat', hue = 'airline_sentiment', data = df0[v1])

In [ ]:
df2 = df0[['airline_sentiment', 'lat', 'lon', 'airline', 'text']][v1].round(decimals = 0)
df2['n'] = 1
df2['text'] = df2['airline'] + ' - ' + df2['text']
df2.drop(columns = 'airline', inplace = True)
df2 = df2.groupby(['airline_sentiment', 'lat', 'lon']).agg({'n': 'count', 'text': list}).reset_index()
df2['n'] = df2['n'].astype(float)

In [ ]:
m = folium.Map(location = [20, 0], tiles = 'OpenStreetMap', zoom_start = 2.5) # 'Mapbox Bright', 'Mapbox Control Room'
for i in df2.index:
    folium.CircleMarker(
        location = [df2.loc[i, 'lat'], df2.loc[i, 'lon']],
        popup = df2.loc[i, 'text'],
        radius = df2.loc[i, 'n'],
        color = palette[df2.loc[i, 'airline_sentiment']],
        fill = True,
        fill_color = palette[df2.loc[i, 'airline_sentiment']]
    ).add_to(m)
m

Scatterplot matrix of numeric and quasi-numeric variables

In [ ]:
#sns.pairplot(df0.sort_values('airline_sentiment'))
sns.pairplot(df0, hue = 'airline_sentiment', hue_order = palette.keys(), palette = palette)

In [ ]:
for x in ['tweet_coord_f', 'tweet_location_f', 'user_timezone_f']:
    df0[x] = df0[x].astype('category')

Analysis of numeric variables

In [ ]:
numeric_variables = list(df0.select_dtypes(include = 'number'))
#numeric_variables = list(df0.select_dtypes(include = ['number', 'datetimetz']))
#sample_df, _ = train_test_split(df0, train_size = sample_n, random_state = seed, stratify = df0[['airline', 'airline_sentiment']])
for y in numeric_variables:
    #sns.catplot(x = 'airline_sentiment', y = y, kind = 'swarm', data = sample_df)
    #sns.catplot(x = 'airline_sentiment', y = y, hue = 'airline_sentiment', kind = 'bar', data = df0.sort_values('airline_sentiment'))
    sns.catplot(x = 'airline_sentiment', y = y, kind = 'boxen', data = df0, order = palette.keys(), palette = palette)

Analysis of categorical variables

In [ ]:
df0['UTC_day_of_week'] = df0['UTC_day_of_week'].astype('category')
categorical_variables = sorted(set(df0.select_dtypes(include = ['object', 'category'])) - set(['UTC_date', 'airline_sentiment', 'airline_sentiment_gold', 'negativereason', 'negativereason_gold', 'text', 'name', 'tweet_coord', 'tweet_location']), key = str.lower)
charts_d = {}
for y in categorical_variables:
    #print(y)
    df5 = pd.crosstab(df0[y], df0['airline_sentiment'], normalize = 0).reset_index().sort_values('negative', ascending = False)
    df5[y] = df5[y].astype('category')
    charts_d[y] = {}
    charts_d[y]['chart0'], charts_d[y]['ax0'] = plt.subplots()
    sns.barplot(y = y, x = 'value', data = df5.melt(id_vars = y), order = df5[y], \
                  hue = 'airline_sentiment', hue_order = palette.keys(), palette = palette)
    charts_d[y]['ax0'].set_title(y)
    charts_d[y]['chart1'], charts_d[y]['ax1'] = plt.subplots()
    sns.countplot(y = y, data = df0, order = df5[y], hue = 'airline_sentiment', hue_order = palette.keys(), palette = palette)

Word clouds

In [ ]:
df0['text2'] = df0['text'].str.strip() \
    .str.replace('(\s*RT @\w+: )|(\s+http\S+$)', '', regex = True) \
    .str.replace('\s*@\w+\s*', ' user ', regex = True) \
    .str.replace('\s*http\S+\s*', ' website ', regex = True) \
    .str.replace('\s*&amp;\s*', ' and ', regex = True)

for x in df0['airline'].unique():
    v2 = df0['airline'] == x
    for y in x.split(' '):
        df0['text2'][v2] = df0['text2'][v2].str.replace('\s*{}\s*'.format(y), ' airline ', regex = True)

df0['text2'] = df0['text2'].str.strip().str.lower()

In [ ]:
for k in ['airline_sentiment', 'airline', 'negativereason']:
    print('By sentiment and {}:'.format(k))
    for x in [x for x in palette.keys() if x in df0[df0[k].notna()]['airline_sentiment'].unique()]:
        for y in sorted(df0[k][df0[k].notna()].unique()):
            if sum((df0['airline_sentiment'] == x) & (df0[k] == y)) > 0:
                plt.figure(figsize = (20, 20))
                wordcloud = WordCloud(
                                      background_color = 'black',
                                      stopwords = set(STOPWORDS).union({'flight', 'user', 'website', 'airline'}),
                                      max_words = 1000,
                                      max_font_size = 110,
                                      random_state = seed
                                    ).generate(str(df0['text2'][(df0['airline_sentiment'] == x) & (df0[k] == y)]))
                plt.imshow(wordcloud)
                plt.title(' '.join([x, y]), fontsize = 20)
                plt.axis('off')
                plt.show()